In [ ]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO
import gradio as gr

IMAGE_URL = "https://cdn.pixabay.com/photo/2023/12/13/22/29/young-woman-8447841_1280.jpg"

ENDPOINT_URL = "https://7ai015-vision.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
API_KEY = "EpqbkwbuaMO7g1gmWgsx8mGN2x8yLRIlt7MxK1TUjXgCZnzAtNYCJQQJ99BGACYeBjFXJ3w3AAAFACOGlNDi"


def random_color():
    # 랜덤한 RGB 색상 튜플 반환
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 


def get_font():
    # OS별로 적절한 폰트 객체 반환 (한글 지원)
    from PIL import ImageFont
    import platform
    
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux      
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 폰트 파일이 없을 경우 기본 폰트 사용
        return ImageFont.load_default(size=font_size)
    

# features 값에 따라 response가 변함
# features가 caption 일 때 gender-neutral-caption을 추가해줘야 한다. 아예 빠지던지, 추가되던지 해야 함.
# features smartCrops 일 때,  smartcrops-aspect-ratios 를 추가해줘야 한다. 빈 값도 가능

def request_image_analysis(image_url, features=["objects"], **kwargs):
    ## ** => dictionary / * => array
    endpoint="{}".format(ENDPOINT_URL)

    params={
        "api-version": "2024-02-01",
        "features": ",".join(features)
    }

    headers={
        "Ocp-Apim-Subscription-Key": API_KEY
    }

    body={
        "url": image_url
    }

    if kwargs is not None: 

        if ("caption" in features or "denseCaptions in features") and "gender_neutral_caption" in kwargs.keys():
            params.update({
                "gender-neutral-caption": kwargs['gender_neutral_caption']
            })

        if "smartCrops" in features and "smartcrops_aspect_ratios" in kwargs.keys():
            params.update({
                "smartcrops-aspect-ratios": kwargs["smartcrops_aspect_ratios"]
            })


    response = requests.post(endpoint, params=params, headers=headers, json=body)

    print(response)

    if response.status_code != 200:
        return None
    
    response_json = response.json()

    return response_json


def draw_image(image_url, features, data):
    # 이미지를 받아온다. 
    # request.get("이미지 URL") = 브라우저에서 이미지 경로 열기
    image_response = requests.get(image_url)

    # 이미지를 열어준다.
    image = Image.open(BytesIO(image_response.content))

    # 이미지 위에 사각형 그리기. 
    draw = ImageDraw.Draw(image)
    font = get_font()
    
    for feature in features:

        if feature == 'tags' or feature == 'caption':
            continue

        feature_key = '{}Result'.format(feature)
        print(feature_key)

        # 객체 인식 결과 리스트
        result_object = data[feature_key]
        block_list = result_object['values']

        color = random_color() # 랜덤 색상

        for block in block_list:

            bounding_box = block['boundingBox']
            x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']

            if feature == 'objets':
                tag = block['tags'][0]
                name = tag['name']
                confidence = tag['confidence']

                formatted_text = "{} ({:.2f}%)".format(name, confidence * 100)
            elif feature == "denseCaptions":
                formatted_text = block['text']
            else:
                formatted_text = ""

            # 좌측 상단, 우측 하단
            # BoundingBox 객체에서 좌표를 가져와서 사각형을 그린다.
            draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)

            # feature의 텍스트를 그려주는 부분
            feature_text_bbox = draw.textbbox((x, y), feature, font=font)
            draw.rectangle([(feature_text_bbox[0], feature_text_bbox[1]), (feature_text_bbox[2], feature_text_bbox[3])], outline=color, fill=color, width=2)
            draw.text((x, y), feature, fill='black', font=font)

            # 텍스트를 그려주는 부분
            if formatted_text:
                text_bbox = draw.textbbox((x, y + 20), formatted_text, font=font)
                draw.rectangle([(text_bbox[0], text_bbox[1]), (text_bbox[2] + 5, text_bbox[3])], outline=color, fill=color, width=2)
                draw.text((x + 5, y + 20),  formatted_text, fill='black', font=font)

    return image


# theme = gr.themes.Soft(primary_hue=gr.themes.colors, indigo)
with gr.Blocks(theme=gr.themes.Soft()) as demo:

    FEATURES = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]

    def click_send(image_url, features):
        print(features)
        response_data = request_image_analysis(image_url, features)
        image = draw_image(image_url, features, response_data)
        # TODO: response_data에서 받은 caption, tags 정보를 가공해서, text로 만든 뒤, return 해준다.
        
        return image


    def change_features(features):
        print(features)
        return features
    
    
    def change_gender(is_neutral):
        print(is_neutral)
        return is_neutral

    features_checkbox = gr.CheckboxGroup(label="Features", choices=FEATURES)

    # gender_neutral_caption
    gender_radio = gr.Radio(label="성중립성", choices=[("중립", True), ("구분", False)], value=False, visible=True)
    # smartcrops_aspect_ratios
    ratio_textbox = gr.Textbox(label="smartCrops 크기", placeholder="ex) 0.75, 1.2, 1.5", visible=True)

    image_url_textbox = gr.Textbox(label="이미지 URL")
    send_button = gr.Button('전송')

    with gr.Row():
        output_image = gr.Image(scale=3, label="결과 화면", interactive=False, type="pil")
        output_textbox = gr.TextArea(scale=1, label="결과 텍스트")

    send_button.click(click_send, inputs=[image_url_textbox, features_checkbox], outputs=[output_image])
    features_checkbox.change(change_features, inputs=[features_checkbox], outputs=[features_checkbox])
    gender_radio.change(change_gender, inputs=[gender_radio], outputs=[gender_radio])

demo.launch()

# 객체 인식 결과 요청 및 시각화
# features = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]

# response_json2 = request_image_analysis(IMAGE_URL, features, gender_neutral_caption=False, smartcrops_aspect_ratios="0.75, 1.8")

# draw_image(IMAGE_URL, features, response_json2)

# response_json2
# draw_image(IMAGE_URL, response_json2)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


True
False
True
False
